引用自：[使用NDArray来处理数据](http://zh.gluon.ai/chapter_crashcourse/ndarray.html)

# 1 使用NDArray来处理数据

对于机器学习来说，处理数据往往是万事之开头。它包含两个部分：
1. 数据读取，
2. 数据已经在内存中时如何处理。

本章将关注后者。

我们首先介绍 `NDArray`，这是 MXNet 储存和变换数据的主要工具。如果你之前用过`NumPy`，你会发现`NDArray`和`NumPy`的多维数组非常类似。当然，`NDArray`提供更多的功能，首先是CPU和GPU的异步计算，其次是自动求导。这两点使得`NDArray`能更好地支持机器学习。

## 1.2 让我们开始

我们先介绍最基本的功能。如果你不懂我们用到的数学操作也不用担心，例如按元素加法、正态分布；我们会在之后的章节分别从数学和代码编写的角度详细介绍。

我们首先从`mxnet`导入`ndarray`这个包

In [35]:
from mxnet import ndarray as nd

然后我们创建一个 $3$ 行和 $4$ 列的 2D 数组（通常也叫**矩阵**），并且把每个元素初始化成 $0$。

In [37]:
nd.zeros((3, 4))


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 3x4 @cpu(0)>

类似的，我们可以创建数组每个元素被初始化成 $1$。

In [38]:
x = nd.ones((3, 4))
x


[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
<NDArray 3x4 @cpu(0)>

或者从 python 的数组直接构造

In [39]:
nd.array([[1, 3], [4, 6]])


[[1. 3.]
 [4. 6.]]
<NDArray 2x2 @cpu(0)>

我们经常需要创建随机数组，即每个元素的值都是随机采样而来，这个经常被用来**初始化模型参数**。以下代码创建数组，它的元素服从均值 $0$ 标准差 $1$ 的正态分布。

In [43]:
y = nd.random_normal(0, 1, shape= (3, 4))
y


[[ 0.47811309  0.9628534   1.7288586   0.42453527]
 [ 1.7228668  -0.32012343 -0.9782293   2.0675592 ]
 [-1.716801    0.47031024  0.2434513   0.30026802]]
<NDArray 3x4 @cpu(0)>

In [44]:
y1 = nd.random_normal(0, 1, shape= (3, 4))
y1


[[-0.011113   -0.43647185  0.53347087  0.4828664 ]
 [ 0.07230883 -0.11204046  1.1557509  -0.17782156]
 [ 2.4579794  -0.5264283  -0.4695239  -0.0990693 ]]
<NDArray 3x4 @cpu(0)>

In [45]:
id(y) == id(y1)  # 值已经改变

False

跟 `NumPy` 一样，每个数组的形状可以通过 `.shape` 来获取：

In [46]:
y.shape

(3, 4)

它的大小，就是总元素个数，是形状的累乘

In [47]:
y.size

12

## 1.3 操作符

NDArray 支持大量的数学操作符：

### 1.3.1 按元素计算

In [48]:
x + y            # 加法


[[ 1.478113    1.9628534   2.7288585   1.4245353 ]
 [ 2.7228668   0.67987657  0.02177072  3.0675592 ]
 [-0.71680105  1.4703102   1.2434514   1.300268  ]]
<NDArray 3x4 @cpu(0)>

In [49]:
x * y        # 乘法


[[ 0.47811309  0.9628534   1.7288586   0.42453527]
 [ 1.7228668  -0.32012343 -0.9782293   2.0675592 ]
 [-1.716801    0.47031024  0.2434513   0.30026802]]
<NDArray 3x4 @cpu(0)>

In [51]:
nd.exp(y)    # 指数运算


[[1.6130279  2.6191592  5.634219   1.5288798 ]
 [5.600561   0.72605944 0.37597626 7.905504  ]
 [0.17963989 1.6004907  1.2756442  1.3502207 ]]
<NDArray 3x4 @cpu(0)>

也可以转置一个矩阵然后计算矩阵乘法，即：

### 1.3.2 矩阵运算

In [53]:
nd.dot(x, y.T)


[[ 3.5943604   2.4920733  -0.70277154]
 [ 3.5943604   2.4920733  -0.70277154]
 [ 3.5943604   2.4920733  -0.70277154]]
<NDArray 3x3 @cpu(0)>

## 1.4 广播（Broadcasting）

当二元操作符左右两边 ndarray 形状不一样时，系统会尝试将其复制到一个共同的形状。例如 `a` 的第 $0$ 维是 $3$, `b` 的第 $0$ 维是 $1$，那么`a+b` 时会将 `b` 沿着第 $0$ 维复制 $3$ 遍：

In [54]:
a = nd.arange(3).reshape((3,1))
b = nd.arange(2).reshape((1,2))

In [55]:
a


[[0.]
 [1.]
 [2.]]
<NDArray 3x1 @cpu(0)>

In [56]:
b


[[0. 1.]]
<NDArray 1x2 @cpu(0)>

In [57]:
a + b


[[0. 1.]
 [1. 2.]
 [2. 3.]]
<NDArray 3x2 @cpu(0)>

## 1.5 跟 NumPy 的转换

ndarray 可以很方便地同 numpy 进行转换

In [58]:
import numpy as np
x = np.ones((2,3))
y = nd.array(x)  # numpy -> mxnet
z = y.asnumpy()  # mxnet -> numpy
print([z, y])

[array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32), 
[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>]


## 1.6 替换操作

在前面的样例中，我们为每个操作新开内存来存储它的结果。例如，如果我们写 `y = x + y`, 我们会把 `y` 从现在指向的实例转到新建的实例上去。我们可以用 Python 的`id()`函数来看这个是怎么执行的：

In [59]:
x = nd.ones((3, 4))
y = nd.ones((3, 4))

before = id(y)
y = y + x
id(y) == before

False

我们可以把结果通过 `[:]` 写到一个之前开好的数组里：

In [60]:
z = nd.zeros_like(x)
z


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 3x4 @cpu(0)>

In [61]:
before = id(z)
z[:] = x + y
id(z) == before

True

In [62]:
z


[[3. 3. 3. 3.]
 [3. 3. 3. 3.]
 [3. 3. 3. 3.]]
<NDArray 3x4 @cpu(0)>

In [64]:
z[:] = 7
z


[[7. 7. 7. 7.]
 [7. 7. 7. 7.]
 [7. 7. 7. 7.]]
<NDArray 3x4 @cpu(0)>

In [66]:
id(z) == before

True

观上述信息，可以知道 `[:]` 操作只将原对象就地修改，即仅仅修改对象的标签（标识符）。

我们还可以为 `x+y` 创建临时空间，然后再复制到 `z`。为了避免浪费内存开销，我们可以使用操作符的全名版本中的 `out` 参数：

In [73]:
nd.elemwise_add(x, y, out= z)


[[3. 3. 3. 3.]
 [3. 3. 3. 3.]
 [3. 3. 3. 3.]]
<NDArray 3x4 @cpu(0)>

In [74]:
id(z) == before

True

如果现有的数组不会复用，我们也可以用 `x[:] = x + y` ，或者 `x += y` 达到这个目的：

In [75]:
before = id(x)
x += y
id(x) == before

True

## 1.7 截取（Slicing）

NXNet NDArray 提供了各种截取方法。截取 x 的 index 为 $1, 2$ 的列：

In [77]:
x = nd.arange(0,9).reshape((3,3))
print('x: ', x)
x[1:3]

x:  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>



[[3. 4. 5.]
 [6. 7. 8.]]
<NDArray 2x3 @cpu(0)>

以及直接写入指定位置：

In [78]:
x[1,2] = 9.0
x


[[0. 1. 2.]
 [3. 4. 9.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>

多维截取:

In [79]:
x = nd.arange(0,9).reshape((3,3))
print('x: ', x)
x[1:2, 1:3]

x:  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>



[[4. 5.]]
<NDArray 1x2 @cpu(0)>

多维写入：

In [80]:
x[1:2, 1:3] = 9.0
x


[[0. 1. 2.]
 [3. 9. 9.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>

## 总结

ndarray 模块提供一系列多维数组操作函数。所有函数列表可以参见[NDArray API文档](https://mxnet.incubator.apache.org/api/python/ndarray.html)。

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/745)

## 小灶

In [86]:
# define a nd.array like np.array
a = mx.nd.random_exponential(shape=(2, 3))   # 指数分布
b = mx.nd.ones((3, 4), dtype= np.int32)
c = mx.nd.full((2, 3), 3) # get a 2x3 matrix with every item 3
d = mx.nd.array([[2, 3], [5, 9]], ctx=mx.gpu(0))

In [87]:
a


[[0.9848373  2.1704154  0.43745184]
 [0.1908379  0.12720822 0.11228119]]
<NDArray 2x3 @cpu(0)>

In [83]:
c


[[3. 3. 3.]
 [3. 3. 3.]]
<NDArray 2x3 @cpu(0)>

In [84]:
d


[[2. 3.]
 [5. 9.]]
<NDArray 2x2 @gpu(0)>

In [90]:
d.context

gpu(0)

In [88]:
# show it implicity
b.asnumpy()

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [89]:
b.context

cpu(0)

In [91]:
a = mx.nd.ones((100,100), mx.cpu())
b = mx.nd.ones((100,100), mx.gpu())
c = mx.nd.ones((100,100), mx.gpu())
a.copyto(c)  # copy from CPU to GPU
d = a.as_in_context(c.context)  # same to above